In [2]:
from fredapi import Fred
import yfinance as yf

import pandas as pd
import numpy as np

import requests
from io import StringIO
from string import Template

from datetime import datetime, timezone
import pytz
from pandas.tseries.offsets import MonthEnd
import pandas_market_calendars as mcal

## Remember your API keys!

In [ ]:
polygonapi = "PPALKDJ62Y8781AA"
fredapi = "AOIDLAKEJF873LAJ"

# Data set #1 - Recent data

<b>Tickers</b>: semiconductor/AI + S&P500\
<b>Period</b>: 2 most recent years\
<b>Periodicity</b>: Monthly

### Polygon.io

In [3]:
polygon_template =  Template("""https://api.polygon.io/v2/aggs
                /ticker/$tckr
                /range/$mult/$tmper
                /$start
                /$end
                ?adjusted=true
                &sort=asc
                &apiKey=$polygonapi""")


In [11]:
polygon_template =  Template("""https://api.polygon.io/v2/aggs/ticker/$tckr/range/$mult/$tmper/$start/$end?adjusted=true&sort=asc&apiKey=$polygonapi""")

#### Polygon.io Call

In [168]:
single_day = polygon_template.substitute(
    tckr='AAPL',
    mult=1,
    tmper='day',
    start='2024-11-29',
    end='2024-11-29',
    polygonapi = polygonapi
)

In [169]:
print(single_day)

https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2024-11-29/2024-11-29?adjusted=true&sort=asc&apiKey=k0w9tcO6Aho4ApjKFQtCgD4hqPdhpSXN


#### Polygon.io Response

In [170]:
single_day_polygon_response = requests.get(single_day)

In [171]:
single_day_polygon_response

<Response [200]>

In [172]:
single_day_polygon_response.ok

True

#### Polygon.io Payload

In [173]:
single_day_data = single_day_polygon_response.json()
single_day_data

{'ticker': 'AAPL',
 'queryCount': 1,
 'resultsCount': 1,
 'adjusted': True,
 'results': [{'v': 25317735.0,
   'vw': 236.5349,
   'o': 234.805,
   'c': 237.33,
   'h': 237.81,
   'l': 233.97,
   't': 1732856400000,
   'n': 268866}],
 'status': 'OK',
 'request_id': '27167a7d0a8f447badaa40fbb0794d7e',
 'count': 1}

#### Payload Elements

In [174]:
single_day_data["ticker"]

'AAPL'

In [175]:
single_day_data["results"]

[{'v': 25317735.0,
  'vw': 236.5349,
  'o': 234.805,
  'c': 237.33,
  'h': 237.81,
  'l': 233.97,
  't': 1732856400000,
  'n': 268866}]

In [176]:
single_day_results = single_day_data["results"]
for item in single_day_results:
    closing_price = item["c"]
    print(closing_price)

237.33


#### Polygon.io Timestamps

In [177]:
json_timestamp = single_day_results[0]["t"]/1000
print(json_timestamp)

1732856400.0


In [178]:
json_time = datetime.fromtimestamp(single_day_results[0]["t"]/1000)
print(json_time)

2024-11-28 23:00:00


### Solving for Time Zone

In [179]:
utc_time = datetime.fromtimestamp(single_day_results[0]["t"]/1000
                                  , tz=timezone.utc)
print(utc_time)

2024-11-29 05:00:00+00:00


In [180]:
eastern = pytz.timezone('America/New_York')
local_time = utc_time.astimezone(eastern)
date = pd.Timestamp(local_time.date(), tz=eastern)
print(date)

2024-11-29 00:00:00-05:00


In [181]:
### Solving for Month End
date = date + MonthEnd(0)
print(date)

2024-11-30 00:00:00-05:00


### Solving for trading day

In [194]:
nyse = mcal.get_calendar("NYSE")
schedule = nyse.schedule(start_date='2024-11-25', end_date='2024-12-05')
trading_days = mcal.date_range(schedule, frequency="1D")
last_trading_day = trading_days[trading_days <= date].max().date()
print(last_trading_day)

2024-11-29


In [191]:
trading_days

DatetimeIndex(['2024-11-25 21:00:00+00:00', '2024-11-26 21:00:00+00:00',
               '2024-11-27 21:00:00+00:00', '2024-11-29 18:00:00+00:00',
               '2024-12-02 21:00:00+00:00', '2024-12-03 21:00:00+00:00',
               '2024-12-04 21:00:00+00:00', '2024-12-05 21:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

### All together

In [196]:
### Setup time zone and dates
eastern = pytz.timezone('America/New_York')
nyse = mcal.get_calendar("NYSE")
schedule = nyse.schedule(start_date='2024-11-25'
                         , end_date='2024-12-05')
trading_days = mcal.date_range(schedule, frequency="1D")

### Polygon.io time element to last trading day
utc_time = datetime.fromtimestamp(single_day_results[0]["t"]/1000
                                  , tz=timezone.utc)
local_time = utc_time.astimezone(eastern)
date = pd.Timestamp(local_time.date(), tz=eastern)
date = date + MonthEnd(0)
last_trading_day = trading_days[trading_days <= date].max().date()
print(last_trading_day)

2024-11-29


In [24]:
# Populating the template
daily_polygon_request = polygon_template.substitute(
    tckr='AAPL',
    mult=1,
    tmper='day',
    start='2024-08-30',
    end='2024-10-01',
    polygonapi=polygonapi
)

eastern = pytz.timezone('America/New_York')

print(daily_polygon_request)

https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2024-08-30/2024-10-01?adjusted=true&sort=asc&apiKey=k0w9tcO6Aho4ApjKFQtCgD4hqPdhpSXN


In [25]:
daily_polygon_response = requests.get(daily_polygon_request)
ticker= daily_polygon_response.json()["ticker"]
daily_data = daily_polygon_response.json()['results']
parsed_data = []
for item in daily_data:
    utc_time = datetime.utcfromtimestamp(item["t"] / 1000)
    date = utc_time.replace(tzinfo=pytz.utc).astimezone(eastern).strftime('%Y-%m-%d')
    closing_price = item["c"]
    parsed_data.append({"ticker": ticker, "date": date, "close": closing_price})

daily_df = pd.DataFrame(parsed_data)
print(daily_df)

   ticker        date   close
0    AAPL  2024-08-30  229.00
1    AAPL  2024-09-03  222.77
2    AAPL  2024-09-04  220.85
3    AAPL  2024-09-05  222.38
4    AAPL  2024-09-06  220.82
5    AAPL  2024-09-09  220.91
6    AAPL  2024-09-10  220.11
7    AAPL  2024-09-11  222.66
8    AAPL  2024-09-12  222.77
9    AAPL  2024-09-13  222.50
10   AAPL  2024-09-16  216.32
11   AAPL  2024-09-17  216.79
12   AAPL  2024-09-18  220.69
13   AAPL  2024-09-19  228.87
14   AAPL  2024-09-20  228.20
15   AAPL  2024-09-23  226.47
16   AAPL  2024-09-24  227.37
17   AAPL  2024-09-25  226.37
18   AAPL  2024-09-26  227.52
19   AAPL  2024-09-27  227.79
20   AAPL  2024-09-30  233.00
21   AAPL  2024-10-01  226.21


C:\Users\ThaddeusRyan.000\AppData\Local\Temp\ipykernel_16564\3320351770.py:6: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  utc_time = datetime.utcfromtimestamp(item["t"] / 1000)


In [212]:
def polygon_eom(ticker, start, end):
    '''
    Gets month-end ticker price from polygon.io
    ticker: string, valid stock symbol
    start: date
    end: date
    '''
    ## setup
    parsed_data = []
    polygon_template = Template("https://api.polygon.io/v2/aggs/ticker/$tckr/range/$mult/$tmper/$start/$end?adjusted=true&sort=asc&apiKey=$polygonapi")
    ## timezone
    eastern = pytz.timezone('America/New_York')
    
    ## string sub into template
    polygon_request = polygon_template.substitute(
                tckr=ticker, mult=1,
                tmper='month', start=start,
                end=end, polygonapi=polygonapi)
    ## request
    polygon_response = requests.get(polygon_request)
    ## response / payload
    if polygon_response.ok == True:
        # try:
        ticker = polygon_response.json()["ticker"]
        data = polygon_response.json()['results']
        for item in data:
            utc_time = datetime.fromtimestamp(item["t"] / 1000, tz=timezone.utc)
            local_time = utc_time.astimezone(eastern)
            date = pd.Timestamp(local_time.date(), tz=eastern) + MonthEnd(0)
            closing_price = item["c"]
            parsed_data.append({"ticker": ticker, "date": date.strftime('%Y-%m-%d'), "close": closing_price})
        parsed_df = pd.DataFrame(parsed_data)
        return parsed_df
        # except KeyError:
        #     print("Symbol %s was not found" %ticker)
    else:
        print('Failed on symbol: %s on response %s' %(ticker, str(polygon_response)))

In [ ]:
def polygon_eom(ticker, start, end):
    '''
    Gets month-end ticker price from polygon.io
    ticker: string, valid stock symbol
    start: date
    end: date
    '''
    ## setup
    parsed_data = []
    polygon_template = Template("""https://api.polygon.io/v2/aggs/ticker/$tckr
                                /range/$mult/$tmper/$start/$end
                                ?adjusted=true&sort=asc&apiKey=$polygonapi""")
    ## timezone
    eastern = pytz.timezone('America/New_York')
    
    ## string sub into template
    polygon_request = polygon_template.substitute(
                tckr=ticker, mult=1,
                tmper='month', start=start,
                end=end, polygonapi=polygonapi)
    

In [ ]:
<continuing> def polygon_eom(ticker, start, end):
    ...
    ## request
    polygon_response = requests.get(polygon_request)
    ## response / payload
    if polygon_response.ok == True:
        # try:
        ticker = polygon_response.json()["ticker"]
        data = polygon_response.json()['results']
        for item in data:
            utc_time = datetime.fromtimestamp(item["t"] / 1000
                                              , tz=timezone.utc)
            local_time = utc_time.astimezone(eastern)
            date = pd.Timestamp(local_time.date(), tz=eastern) + MonthEnd(0)
            closing_price = item["c"]
            parsed_data.append({"ticker": ticker
                                , "date": date.strftime('%Y-%m-%d')
                                , "close": closing_price})
        parsed_df = pd.DataFrame(parsed_data)
        return parsed_df
        # except KeyError:
        #     print("Symbol %s was not found" %ticker)
    else:
        print('Failed on symbol: %s on response %s' %(ticker, str(polygon_response)))

In [213]:
nvda = ['NVDA']
all_df = polygon_iter(nvda,'2023-01-01','2024-02-01')

In [73]:
def polygon_iter(tickers, start, end):
    '''
    Iterates over a list of tickers
    ticker: list of string symbols
    start: date
    end: date
    '''
    collected_data = []
    
    for ticker in tickers:
        try:
            t_df = polygon_eom(ticker, start, end)
            if not t_df.empty:
                collected_data.append(t_df)
        except Exception as e:
            print(f"Error processing {ticker}: {e}")
            
    # Combine df from all tickers
    if collected_data:
        combined_df = pd.concat(collected_data, ignore_index=True)
    else:
        combined_df = pd.DataFrame()
    return combined_df

In [227]:
### polygon.io - Go back 2 years, up to 5 symbols
tickerz = ['NVDA','AMD','INTC','MSFT','SPY']
post_df = polygon_iter(tickerz,'2023-01-01','2024-12-31')
post_df.groupby('ticker')['date'].agg('count')

ticker
AMD     24
INTC    24
MSFT    24
NVDA    24
SPY     24
Name: date, dtype: int64

In [228]:
post_df.groupby('ticker')\
            .agg(earliest=('date','min')
                ,latest=('date','max')
                ,lowest=('close','min')
                ,highest=('close','max'))

,earliest,latest,lowest,highest
ticker,,,,
AMD,2023-01-31,2024-12-31,75.150,192.53
INTC,2023-01-31,2024-12-31,20.050,50.25
MSFT,2023-01-31,2024-12-31,247.810,446.95
NVDA,2023-01-31,2024-12-31,19.537,138.25
SPY,2023-01-31,2024-12-31,396.260,602.55


### Pivot the stock data

In [246]:
pivoted_df = post_df.pivot(index="date"
                           , columns="ticker"
                           , values="close")
pivoted_df.head()

ticker,AMD,INTC,MSFT,NVDA,SPY
date,,,,,
2023-01-31,75.15,28.26,247.81,19.537,406.48
2023-02-28,78.58,24.93,249.42,23.216,396.26
2023-03-31,98.01,32.67,288.30,27.777,409.39
2023-04-30,89.37,31.06,307.26,27.749,415.93
2023-05-31,118.21,31.44,328.39,37.834,417.85


### Persist the stock data

In [248]:
pivoted_df.to_csv('scen1_stocks.csv')

In [249]:
### Restore the stock data
scen1_stocks = pd.read_csv('scen1_stocks.csv', index_col='date')
scen1_stocks.head()

,AMD,INTC,MSFT,NVDA,SPY
date,,,,,
2023-01-31,75.15,28.26,247.81,19.537,406.48
2023-02-28,78.58,24.93,249.42,23.216,396.26
2023-03-31,98.01,32.67,288.30,27.777,409.39
2023-04-30,89.37,31.06,307.26,27.749,415.93
2023-05-31,118.21,31.44,328.39,37.834,417.85


In [82]:
polygon_template = Template("https://api.polygon.io/v2/aggs/ticker/$tckr/range/$mult/$tmper/$start/$end?adjusted=true&sort=asc&apiKey=$polygonapi")
eastern = pytz.timezone('America/New_York')

# Populating the template
polygon_request = polygon_template.substitute(
    tckr='AAPL',
    mult=1,
    tmper='month',
    start='2023-10-01',
    end='2024-10-02',
    polygonapi=polygonapi
)

print(polygon_request)

https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/month/2023-10-01/2024-10-02?adjusted=true&sort=asc&apiKey=k0w9tcO6Aho4ApjKFQtCgD4hqPdhpSXN


In [83]:
### Call the request
polygon_response = requests.get(polygon_request)

In [84]:
parsed_data = []
ticker = polygon_response.json()["ticker"]
data = polygon_response.json()['results']
for item in data:
    utc_time = datetime.fromtimestamp(item["t"] / 1000, tz=timezone.utc)
    local_time = utc_time.astimezone(eastern)
    date = pd.Timestamp(local_time.date(), tz=eastern) + MonthEnd(0)
    closing_price = item["c"]
    parsed_data.append({"ticker": ticker, "date": date.strftime("%Y-%m-%d"), "close": closing_price})

df = pd.DataFrame(parsed_data)
print(df)

   ticker        date   close
0    AAPL  2023-10-31  170.77
1    AAPL  2023-11-30  189.95
2    AAPL  2023-12-31  192.53
3    AAPL  2024-01-31  184.40
4    AAPL  2024-02-29  180.75
5    AAPL  2024-03-31  171.48
6    AAPL  2024-04-30  170.33
7    AAPL  2024-05-31  192.25
8    AAPL  2024-06-30  210.62
9    AAPL  2024-07-31  222.08
10   AAPL  2024-08-31  229.00
11   AAPL  2024-09-30  233.00
12   AAPL  2024-10-31  225.91


### FRED

In [3]:
fred = Fred(api_key = fredapi)

In [5]:
### Look for industrial production
x = fred.search('industrial')
x.head(3)

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
INDPRO,INDPRO,2025-01-11,2025-01-11,Industrial Production: Total Index,1919-01-01,2024-11-01,Monthly,M,Index 2017=100,Index 2017=100,Seasonally Adjusted,SA,2024-12-17 08:26:14-06:00,79,explanatory notes (https://www.federalreserve....
IPB50001N,IPB50001N,2025-01-11,2025-01-11,Industrial Production: Total Index,1919-01-01,2024-11-01,Monthly,M,Index 2017=100,Index 2017=100,Not Seasonally Adjusted,NSA,2024-12-17 08:26:38-06:00,39,explanatory notes (https://www.federalreserve....
IPB50001SQ,IPB50001SQ,2025-01-11,2025-01-11,Industrial Production: Total Index,1919-01-01,2024-07-01,Quarterly,Q,Index 2017=100,Index 2017=100,Seasonally Adjusted,SA,2024-12-17 08:26:52-06:00,24,explanatory notes (https://www.federalreserve....


In [7]:
### Look for potential inflation
x = fred.search('potential inflation consumer')
x.head(3)

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
CPIEALL,CPIEALL,2025-01-11,2025-01-11,Research Consumer Price Index: All Items,1982-12-01,2024-11-01,Monthly,M,Index 1982=100,Index 1982=100,Seasonally Adjusted,SA,2024-12-11 15:31:02-06:00,32,The BLS calculates a research price index call...
CPIEHOUSE,CPIEHOUSE,2025-01-11,2025-01-11,Research Consumer Price Index: Housing,1982-12-01,2024-11-01,Monthly,M,Index 1982=100,Index 1982=100,Seasonally Adjusted,SA,2024-12-11 15:31:01-06:00,32,The BLS calculates a research price index call...
CPIEMEDCARE,CPIEMEDCARE,2025-01-11,2025-01-11,Research Consumer Price Index: Medical Care,1982-12-01,2024-11-01,Monthly,M,Index 1982=100,Index 1982=100,Seasonally Adjusted,SA,2024-12-11 15:31:02-06:00,12,The BLS calculates a research price index call...


In [9]:
### Look for risk free 1-year rate 
x = fred.search("risk free 1-Year")
x.head()

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
DGS1,DGS1,2025-01-11,2025-01-11,Market Yield on U.S. Treasury Securities at 1-...,1962-01-02,2025-01-09,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2025-01-10 15:21:48-06:00,79,H.15 Statistical Release notes (https://www.fe...
WGS1YR,WGS1YR,2025-01-11,2025-01-11,Market Yield on U.S. Treasury Securities at 1-...,1962-01-05,2025-01-03,"Weekly, Ending Friday",W,Percent,%,Not Seasonally Adjusted,NSA,2025-01-06 15:16:53-06:00,56,H.15 Statistical Release notes (https://www.fe...
GS1,GS1,2025-01-11,2025-01-11,Market Yield on U.S. Treasury Securities at 1-...,1953-04-01,2024-12-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2025-01-02 16:44:46-06:00,55,H.15 Statistical Release notes (https://www.fe...


In [10]:
### industrial production
ind = fred.get_series('INDPRO')
### potential inflation
inf = fred.get_series('CPIEALL')
### risk-free rate
risk_free = fred.get_series('GS1')

In [34]:
### set Fred series to same months
### dates are first day for whole month
ind_1 = ind['2023-01-01':'2024-12-01']
inf_1 = inf['2023-01-01':'2024-12-01']
rf_1 = risk_free['2023-01-01':'2024-12-01']

In [71]:
### evaluate industrial prod data
ind_0 = ind_1.reset_index()\
    .rename({'index':'month'
             , 0:'industrial_prod'}
            ,axis=1)
ind_0.head()

,month,industrial_prod
0,2023-01-01,102.7476
1,2023-02-01,102.8003
2,2023-03-01,102.8143
3,2023-04-01,103.2241
4,2023-05-01,102.9809


In [250]:
### evaluate inflation data
inf_0 = inf_1.reset_index()\
    .rename({'index':'month'
             , 0:'potential_inflation'}
            ,axis=1)
inf_0.head()

,month,potential_inflation
0,2023-01-01,325.520
1,2023-02-01,327.499
2,2023-03-01,328.457
3,2023-04-01,329.868
4,2023-05-01,330.663


In [252]:
### evaluate risk-free
rf_0 = rf_1.reset_index()\
    .rename({'index':'month'
             , 0:'risk_free'}
            ,axis=1)
rf_0.head()

,month,risk_free
0,2023-01-01,4.69
1,2023-02-01,4.93
2,2023-03-01,4.68
3,2023-04-01,4.68
4,2023-05-01,4.91


In [256]:
### Align to a single dataframe
scen1_fred = inf_0.merge(ind_0, left_index=True, right_index=True)\
                .merge(rf_0, left_index=True, right_index=True)
scen1_fred = scen1_fred[['month','potential_inflation'
                        ,'industrial_prod','risk_free']]
scen1_fred.head()

,month,potential_inflation,industrial_prod,risk_free
0,2023-01-01,325.520,102.7476,4.69
1,2023-02-01,327.499,102.8003,4.93
2,2023-03-01,328.457,102.8143,4.68
3,2023-04-01,329.868,103.2241,4.68
4,2023-05-01,330.663,102.9809,4.91


In [258]:
### align to final day of month
scen1_fred['month'] = scen1_fred['month'].apply(lambda x: x + MonthEnd(0))
scen1_fred.tail()

,month,potential_inflation,industrial_prod,risk_free
18,2024-07-31,343.324,102.5340,4.90
19,2024-08-31,343.643,103.0831,4.43
20,2024-09-30,344.322,102.5283,4.03
21,2024-10-31,344.837,102.1123,4.20
22,2024-11-30,344.803,101.9621,4.33


In [260]:
### persist data
scen1_fred.to_csv("scen1_fred.csv")

### read back in as a test
scen1_fred_test = pd.read_csv("scen1_fred.csv", index_col='month')
scen1_fred_test.head()

,Unnamed: 0,potential_inflation,industrial_prod,risk_free
month,,,,
2023-01-31,0,325.520,102.7476,4.69
2023-02-28,1,327.499,102.8003,4.93
2023-03-31,2,328.457,102.8143,4.68
2023-04-30,3,329.868,103.2241,4.68
2023-05-31,4,330.663,102.9809,4.91
